In [2]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB 495.5 kB/s eta 0:00:30
   ---------------------------------------- 0.1/14.6 MB 1.1 MB/s eta 0:00:14
    --------------------------------------- 0.2/14.6 MB 1.4 MB/s eta 0:00:10
    --------------------------------------- 0.2/14.6 MB 1.4 MB/s eta 0:00:10
   - -------------------------------------- 0.5/14.6 MB 2.1 MB/s eta 0:00:07
   - -------------------------------------- 0.7/14.6 MB 2.1 MB/s eta 0:00:07
   -- ------------------------------------- 0.9/14.6 MB 2.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.1/14.6 MB 2.6 MB/s eta 0:00:06
   --- ------------------------------------ 1.3/14.6 MB 2.8 MB/s eta 0:00:05
   ---- ----------------------------------- 1.5/14.6 MB 3.0 MB/s eta 0:00:05
   ---- ----------------------------------- 1.7/14.6 MB 3.1 MB/s eta 0:00:05
   ----- --

In [4]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [6]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [13]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [29]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
vectors.shape

C:\Users\durge\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(8, 768)

In [31]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [36]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [39]:
index.add(vectors)

In [41]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000208828277E0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [53]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [55]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [57]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [74]:
distances, I = index.search(svec, k=2)
distances

array([[1.384484 , 1.4039094]], dtype=float32)

In [76]:
I

array([[3, 2]], dtype=int64)

In [78]:
I.tolist()

[[3, 2]]

In [80]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [82]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [84]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query